In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pdfplumber

In [3]:
import pdfplumber

def pdf_2_dict(pdf_path):
    # Extract text from pdf file
    with pdfplumber.open(pdf_path) as temp:
        first_page = temp.pages[0]
        text = first_page.extract_text()
        text_list = text.split("\n")

    # Create empty dict
    pdf_dict = {}

    # Generate new fields
    pdf_dict["id"] = int(text_list[1].split()[1])
    pdf_dict["facilityName"] = " ".join(text_list[2].split()[2:-1])
    pdf_dict["FacilityInspireID"] = text_list[3].split()[1]
    pdf_dict["countryName"] = " ".join(text_list[4].split("CONTINENT:")[0].split()[1:])
    pdf_dict["CONTINENT"] = " ".join(text_list[4].split("CONTINENT:")[1:])[1:]
    pdf_dict["City"] = " ".join(text_list[5].split()[1:])
    pdf_dict["EPRTRSectorCode"] = int(" ".join(text_list[6].split("eprtrSectorName:")[0].split()[1:]))
    pdf_dict["eprtrSectorName"] = " ".join(text_list[6].split("eprtrSectorName:")[1:])[1:]
    pdf_dict["EPRTRAnnexIMainActivityCode"] = text_list[7].split(":")[1]
    pdf_dict["targetRelease"] = " ".join(text_list[8].split("pollutant:")[0].split()[1:])
    pdf_dict["pollutant"] = " ".join(text_list[8].split("pollutant:")[1:])[1:]
    pdf_dict["emissions"] = float(text_list[9].split()[1].replace(",", "."))
    pdf_dict["DAY"] = int(text_list[10].split()[1])
    pdf_dict["MONTH"] = int(text_list[10].split()[3])
    pdf_dict["reportingYear"] = int(text_list[10].split()[5])
    pdf_dict["max_wind_speed"] = float(text_list[12].split()[1].replace(",", "."))
    pdf_dict["min_wind_speed"] = float(text_list[12].split()[3].replace(",", "."))
    pdf_dict["avg_wind_speed"] = float(text_list[12].split()[5].replace(",", "."))
    pdf_dict["max_temp"] = float(text_list[13].split()[1].replace(",", "."))
    pdf_dict["min_temp"] = float(text_list[13].split()[3].replace(",", "."))
    pdf_dict["avg_temp"] = float(text_list[13].split()[5].replace(",", "."))
    pdf_dict["DAY WITH FOGS"] = int(text_list[14].split()[-1])
    pdf_dict["REPORTER NAME"] = text_list[15].split("REPORTER NAME:")[-1]
    pdf_dict["CITY ID"] = text_list[16].split("CITY_ID ")[-1]

    return pdf_dict

In [4]:
import pandas as pd

pdf_dict = pdf_2_dict("/content/drive/MyDrive/Data_Science_&_AI/NUWE/pdf/pdfs-1.pdf")
df = pd.Series(pdf_dict)
df

id                                                              81597
facilityName                   Millerhill Recycling & Energy Recovery
FacilityInspireID                          UK.SEPA/200002651.Facility
countryName                                            United Kingdom
CONTINENT                                                      EUROPE
City                                             Millerhill, Dalkeith
EPRTRSectorCode                                                     5
eprtrSectorName                       Waste and wastewater management
EPRTRAnnexIMainActivityCode                                      5(b)
targetRelease                                                     AIR
pollutant                                       Nitrogen oxides (NOX)
emissions                                                    175000.0
DAY                                                                12
MONTH                                                               6
reportingYear       

In [5]:
import os

pdf_path_dir = "/content/drive/MyDrive/Data_Science_&_AI/NUWE/pdf/"
pdf_path_list = os.listdir(pdf_path_dir)

In [9]:
from pandas_profiling import ProfileReport

In [21]:
df = pd.DataFrame()
for pdf_path in pdf_path_list:
    pdf_dict = pdf_2_dict(pdf_path_dir + pdf_path)
    pd_series = pd.Series(pdf_dict)
    df = df.append(pd_series, ignore_index=True)

In [23]:
df = df.set_index("id")
df.index = df.index.astype(int)

df

,facilityName,FacilityInspireID,countryName,CONTINENT,City,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,targetRelease,pollutant,...,reportingYear,max_wind_speed,min_wind_speed,avg_wind_speed,max_temp,min_temp,avg_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
id,,,,,,,,,,,,,,,,,,,,,
81540,"Greengairs Landfill Site, Airdrie",UK.SEPA/200000116.Facility,United Kingdom,EUROPE,Airdrie,5.0,Waste and wastewater management,5(d),AIR,Methane (CH4),...,2019.0,2.110000e+15,2.730000e+16,2.160000e+16,1.700000e+15,2.000000e+16,1.870000e+15,19.0,Zachary Randolph,12ab217b40114f8deef2237c7a4d37ae
81571,"Sullom Voe Terminal CHP, Shetland",UK.SEPA/200001408.Facility,United Kingdom,EUROPE,Mossbank,1.0,Energy sector,1(c),AIR,Nitrogen oxides (NOX),...,2019.0,1.780000e+16,2.490000e+15,1.770000e+15,1.800000e+16,2.000000e+15,1.800000e+16,1.0,Sarah Stone,9a1df6efaa8aae020052840bafbfc7cb
81518,Alloa Glass Factory,UK.SEPA/200000073.Facility,United Kingdom,EUROPE,Alloa,3.0,Mineral industry,3(e),AIR,Carbon dioxide (CO2),...,2019.0,1.110000e+16,2.030000e+16,1.600000e+16,-1.900000e+16,4.070000e+16,1.330000e+15,4.0,Vicki Brown,2cc8f54182c37b8907f534011ea01e6f
81517,Fife Ethylene Plant,UK.SEPA/200000061.Facility,United Kingdom,EUROPE,Cowdenbeath,1.0,Energy sector,1(c),AIR,Nitrogen oxides (NOX),...,2019.0,1.160000e+16,2.180000e+16,1.650000e+16,8.030000e+15,1.040000e+16,8.940000e+15,10.0,Aaron Johnson,3c563ab0d76fc84128574b5da82f769a
81592,Dunbar Energy Recovery Facility,UK.SEPA/200002618.Facility,United Kingdom,EUROPE,"East Barns, Dunbar, East Lothian",5.0,Waste and wastewater management,5(b),AIR,Nitrogen oxides (NOX),...,2019.0,1.930000e+16,2.670000e+15,1.870000e+16,6.820000e+15,1.370000e+16,8.770000e+15,1.0,Christopher Tate,6935acf68ac33c9c85ff634275880d97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81563,DSM @ Drakemyre Chemical Works,UK.SEPA/200000207.Facility,United Kingdom,EUROPE,Dalry,4.0,Chemical industry,4(a)(ii),AIR,Methane (CH4),...,2019.0,1.600000e+16,2.420000e+15,1.860000e+16,3.810000e+16,6.540000e+16,3.590000e+16,10.0,Kathryn Smith,111a1f9724ed88a23488fd6fb36a050e
81560,Caledonian Paper Mill,UK.SEPA/200000176.Facility,United Kingdom,EUROPE,Irvine,6.0,Paper and wood production and processing,6(b),AIR,Carbon dioxide (CO2),...,2019.0,1.730000e+16,2.510000e+16,1.780000e+16,1.520000e+16,1.800000e+16,1.620000e+16,13.0,Joel Foster,3ecc035dd613d1e8c86207e007da26a4
81587,INEOS Infrastructure (Grangemouth) Ltd,UK.SEPA/200002494.Facility,United Kingdom,EUROPE,Grangemouth,1.0,Energy sector,1(c),AIR,Carbon dioxide (CO2),...,2019.0,1.720000e+16,2.360000e+16,1.900000e+15,6.520000e+15,1.150000e+16,9.450000e+15,4.0,Teresa Soto,ce6badb6a385e25ce519dc600eb7402e
